In [1]:
import pandas as pd
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import layers
from keras.layers import Activation, Dense, Dropout, Conv1D, Conv2D, Flatten, BatchNormalization, ZeroPadding2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras import regularizers
from keras.optimizers import SGD
import keras.backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras import regularizers
import random

Using TensorFlow backend.


In [2]:
dataset  = pd.read_pickle('datasetaugment.pickle')
for i,j in enumerate(dataset['mel_spec']):
    if len(j[0])!=434:
         dataset = dataset.drop(i,0)
del dataset['index']
dataset = dataset.sample(frac=1)
dataset = dataset.reset_index()
del dataset['index']
dataset['Genre'] = dataset['Genre'].replace('Classical','classical')
dataset['Genre'] = dataset['Genre'].replace('Country','country')
dataset['Genre'] = dataset['Genre'].replace('Electronic','electronic')
dataset['Genre'] = dataset['Genre'].replace('Hip-Hop','hip-hop')
dataset['Genre'] = dataset['Genre'].replace('Pop','pop')
dataset['Genre'] = dataset['Genre'].replace('Rock','rock')

In [3]:
train = dataset[dataset['Genre']!='country']
test = dataset[:1000]

In [4]:
print('Hip-Hop:   ',len(test[test['Genre']=='hip-hop']))
print('Pop:       ',len(test[test['Genre']=='pop']))
print('Country:   ',len(test[test['Genre']=='country']))
print('Rock:      ',len(test[test['Genre']=='rock']))
print('Electronic:',len(test[test['Genre']=='electronic']))
print('Classical: ',len(test[test['Genre']=='classical']))

Hip-Hop:    207
Pop:        226
Country:    190
Rock:       119
Electronic: 147
Classical:  111


In [16]:
X_train, Y_train = train['mel_spec'],train['Genre']
X_validation, Y_validation = test['mel_spec'],test['Genre']

# Reshape for CNN input

# X_list = [x.reshape(128, 434,1) for x in X_train]
# X1 = np.array(X_list[:3000])
# X2 = np.array(X_list[3000:6000])
# X3 = np.array(X_list[6000:9000])
# X4 = np.array(X_list[9000:12000])
# X5 = np.array(X_list[10000:])
# X_train = np.concatenate((X1,X2,X3,X4,X5))
X_validation = np.array([x.reshape(128, 434, 1) for x in X_validation])

# Glove
wordvec = {}
f=open("glove.6B.100d.txt",encoding='utf-8')
classes = ['classical', 'country', 'electronic', 'hip-hop', 'pop', 'rock']
for i in f:
    word = i.split()[0]
    if word in classes:
        wordvec[word] = i.split()[1:]

        
        
Y_train = np.array([wordvec[i] for i in Y_train])
Y_validation = np.array([wordvec[i] for i in Y_validation])

In [17]:
Y_train.shape

(11636, 100)

In [18]:
tf.keras.backend.clear_session()
model = Sequential()
input_shape=(128, 434, 1)

model.add(Conv2D(24, (4, 4), strides=(1, 1), input_shape=input_shape))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (4, 4), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (4, 4), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(100))
model.add(Activation('linear'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 125, 431, 24)      408       
_________________________________________________________________
average_pooling2d_1 (Average (None, 62, 215, 24)       0         
_________________________________________________________________
activation_1 (Activation)    (None, 62, 215, 24)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 62, 215, 48)       18480     
_________________________________________________________________
average_pooling2d_2 (Average (None, 31, 107, 48)       0         
_________________________________________________________________
activation_2 (Activation)    (None, 31, 107, 48)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 107, 48)       36912     
__________

In [11]:
model = load_model('ZSL_music_classification.h5')

W1016 14:11:17.265329 23632 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\envs\Review\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1016 14:11:17.497342 23632 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\envs\Review\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1016 14:11:17.589348 23632 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\envs\Review\lib\site-packages\keras\backend\tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W1016 14:11:17.672353 23632 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\envs\Review\lib\site-packages\keras\backend\tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1016 14:11:17.673352 23632 de

In [ ]:
yhat = model.predict(X_validation)

In [29]:
# validaiton_embeddings = regression_based.predict(X_validation)

targets_embeddings = [wordvec['country']]

targets_embeddings = np.array(targets_embeddings, dtype=np.float32)

from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import cosine_proximity
from scipy.spatial.distance import cosine


neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(list(wordvec.values()),[0,1,2,3,4,5])
Y_pred_validation = neigh.predict(yhat)

C:\ProgramData\Anaconda3\envs\Review\lib\site-packages\sklearn\utils\validation.py:563: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  FutureWarning)
C:\ProgramData\Anaconda3\envs\Review\lib\site-packages\sklearn\utils\validation.py:563: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  FutureWarning)


In [38]:
genreclass = {k:v for k,v in zip(list(wordvec.keys()),[0,1,2,3,4,5])}
Y_val = [genreclass[i] for i in test['Genre']]
cnt = 0
for i,j in zip(Y_val, Y_pred_validation):
    if i==j:
        cnt+=1
print('accuracy =', cnt/len(Y_val))

accuracy = 0.342


In [27]:
len(list(wordvec.values())[0])

100

In [ ]:
Y_pred_validation